# Neural Network from Scratch Applied to Root Finding

This project focuses on building a neural network from scratch to solve the root-finding problem for quadratic equations. The goal is to accurately predict the real roots of these equations using an optimized neural network architecture. Key features include modular design, performance optimization, and flexibility, enabling the network to tackle root-finding tasks with precision and efficiency.

By combining innovative techniques and a streamlined structure, this model is designed to handle quadratic root predictions with ease.

## Step 1: Modular Neural Network Classes

In [1]:
import numpy as np

class Layer:
    def __init__(self, dim_in, dim_out, activation=None):
        self.weights = 0.1 * np.random.randn(dim_in, dim_out)
        self.biases = np.zeros((1, dim_out))
        self.activation = activation
        self.momentum_weights = np.zeros_like(self.weights)
        self.cache_weights = np.zeros_like(self.weights)
        self.momentum_biases = np.zeros_like(self.biases)
        self.cache_biases = np.zeros_like(self.biases)

    def forward(self, inputs):
        self.inputs = inputs
        self.outputs = np.dot(inputs, self.weights) + self.biases
        if self.activation == 'relu':
            self.activation_outputs = np.maximum(0, self.outputs)
        else:
            self.activation_outputs = self.outputs
        return self.activation_outputs

    def backward(self, grad):
        if self.activation == 'relu':
            grad[self.outputs <= 0] = 0
        self.grad_weights = np.dot(self.inputs.T, grad)
        self.grad_biases = np.sum(grad, axis=0, keepdims=True)
        return np.dot(grad, self.weights.T)

### Explanation
We encapsulated the neural network layer in a `Layer` class, making it easier to manage and reuse. Each layer can perform a forward pass with optional ReLU activation, and backpropagate gradients for updating weights and biases.

## Step 2: The Optimizer Class

In [2]:
class OptimizerAdam:
    def __init__(self, learning_rate=0.01, decay=1e-3, beta1=0.9, beta2=0.999):
        self.learning_rate = learning_rate
        self.decay = decay
        self.beta1 = beta1
        self.beta2 = beta2
        self.iterations = 0

    def update(self, layer):
        if not hasattr(layer, 'momentum_weights'):
            return
        layer.momentum_weights = self.beta1 * layer.momentum_weights + (1 - self.beta1) * layer.grad_weights
        layer.momentum_biases = self.beta1 * layer.momentum_biases + (1 - self.beta1) * layer.grad_biases

        corrected_momentum_weights = layer.momentum_weights / (1 - self.beta1 ** (self.iterations + 1))
        corrected_momentum_biases = layer.momentum_biases / (1 - self.beta1 ** (self.iterations + 1))

        layer.cache_weights = self.beta2 * layer.cache_weights + (1 - self.beta2) * layer.grad_weights ** 2
        layer.cache_biases = self.beta2 * layer.cache_biases + (1 - self.beta2) * layer.grad_biases ** 2

        corrected_cache_weights = layer.cache_weights / (1 - self.beta2 ** (self.iterations + 1))
        corrected_cache_biases = layer.cache_biases / (1 - self.beta2 ** (self.iterations + 1))

        lr = self.learning_rate * (1 / (1 + self.decay * self.iterations))

        layer.weights += -lr * corrected_momentum_weights / (np.sqrt(corrected_cache_weights) + 1e-8)
        layer.biases += -lr * corrected_momentum_biases / (np.sqrt(corrected_cache_biases) + 1e-8)
        self.iterations += 1

### Explanation
Here, we introduce the Adam optimizer as a class, which manages learning rate decay and tracks momentum and cache for weight updates. This makes our code more modular and allows easy substitution with other optimizers if needed.

## Step 3: Data Generation for Quadratic Root Finding

In [3]:
def generate_quadratic_data(samples):
    X, y = [], []
    for _ in range(samples):
        a = np.random.uniform(-100, 100)
        b = np.random.uniform(-100, 100)
        c = np.random.uniform(-100, 100)
        roots = np.roots([a, b, c])
        real_roots = [root.real for root in roots if root.imag == 0]
        if real_roots:
            X.append([a, b, c])
            y.append([max(real_roots)])
    return np.array(X), np.array(y)

X_train, y_train = generate_quadratic_data(3000)
X_test, y_test = generate_quadratic_data(1000)

### Explanation
In this step, we generate synthetic data for training and testing. Each data point represents the coefficients of a quadratic equation and the largest real root of the equation as the target.

## Step 4: Training the Neural Network

In [6]:
network = [Layer(3, 64, activation='relu'), Layer(64, 64, activation='relu'), Layer(64, 1)]
optimizer = OptimizerAdam(learning_rate=0.01, decay=1e-3)

epochs = 10000
for epoch in range(epochs):
    inputs = X_train
    for layer in network:
        inputs = layer.forward(inputs)
    predictions = inputs

    loss = np.mean((predictions - y_train) ** 2)
    grad = (2 * (predictions - y_train)) / y_train.size

    for layer in reversed(network):
        grad = layer.backward(grad)

    for layer in network:
        optimizer.update(layer)

    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {loss:.4f}")

Epoch 0, Loss: 296.5527
Epoch 100, Loss: 94.0541
Epoch 200, Loss: 11.6159
Epoch 300, Loss: 1.4288
Epoch 400, Loss: 0.6062
Epoch 500, Loss: 0.4476
Epoch 600, Loss: 0.3722
Epoch 700, Loss: 0.3309
Epoch 800, Loss: 0.2996
Epoch 900, Loss: 0.2739
Epoch 1000, Loss: 0.2497
Epoch 1100, Loss: 0.2324
Epoch 1200, Loss: 0.2196
Epoch 1300, Loss: 0.2073
Epoch 1400, Loss: 0.1969
Epoch 1500, Loss: 0.1879
Epoch 1600, Loss: 0.1791
Epoch 1700, Loss: 0.1706
Epoch 1800, Loss: 0.1632
Epoch 1900, Loss: 0.1562
Epoch 2000, Loss: 0.1521
Epoch 2100, Loss: 0.1439
Epoch 2200, Loss: 0.1375
Epoch 2300, Loss: 0.1312
Epoch 2400, Loss: 0.1322
Epoch 2500, Loss: 0.1189
Epoch 2600, Loss: 0.1608
Epoch 2700, Loss: 0.1087
Epoch 2800, Loss: 0.1068
Epoch 2900, Loss: 0.1013
Epoch 3000, Loss: 0.0969
Epoch 3100, Loss: 0.0932
Epoch 3200, Loss: 0.0897
Epoch 3300, Loss: 0.0869
Epoch 3400, Loss: 0.0844
Epoch 3500, Loss: 0.0895
Epoch 3600, Loss: 0.1233
Epoch 3700, Loss: 0.0767
Epoch 3800, Loss: 0.0804
Epoch 3900, Loss: 0.0734
Epoch 40

### Explanation
We train the network using our new modular structure. Loss is calculated using Mean Squared Error (MSE), and gradients are backpropagated to update each layer's weights and biases via the Adam optimizer.

## Step 5: Evaluating the Model

In [7]:
inputs = X_test
for layer in network:
    inputs = layer.forward(inputs)
test_predictions = inputs
test_loss = np.mean((test_predictions - y_test) ** 2)
print(f"Test Loss: {test_loss:.4f}")

Test Loss: 93.1237


### Explanation
Finally, we evaluate the trained network on the test set, printing the test loss. The model should be reasonably accurate in predicting the real roots of quadratic equations.

In [8]:

# Solution Real Numpy Test Set: Calculating the real roots using numpy's roots
numpy_real_solutions = []
for coefficients in X_test:
    roots = np.roots(coefficients)
    real_roots = [root.real for root in roots if root.imag == 0]
    numpy_real_solutions.append(max(real_roots) if real_roots else None)

# Filter None values for test set where no real roots were found
filtered_y_test = []
filtered_model_predictions = []
filtered_numpy_solutions = []

for y_true, model_pred, numpy_pred in zip(y_test, test_predictions, numpy_real_solutions):
    if numpy_pred is not None:
        filtered_y_test.append(y_true[0])
        filtered_model_predictions.append(model_pred[0])
        filtered_numpy_solutions.append(numpy_pred)

filtered_y_test = np.array(filtered_y_test)
filtered_model_predictions = np.array(filtered_model_predictions)
filtered_numpy_solutions = np.array(filtered_numpy_solutions)

In [9]:

# Display Solution Real Model Test Set and Solution Real Numpy Test Set for comparison
import pandas as pd

comparison_df = pd.DataFrame({
    'True Solution': filtered_y_test,
    'Model Solution': filtered_model_predictions,
    'Numpy Solution': filtered_numpy_solutions
})

comparison_df.head(10)

,True Solution,Model Solution,Numpy Solution
0,1.647576,1.806480,1.647576
1,3.463067,3.337222,3.463067
2,15.631952,10.920216,15.631952
3,1.171441,1.153938,1.171441
4,0.915359,0.821571,0.915359
5,1.090382,0.953225,1.090382
6,1.620483,1.510341,1.620483
7,0.282316,0.044199,0.282316
8,0.405600,0.423949,0.405600
9,1.020239,1.073542,1.020239


In [10]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(filtered_numpy_solutions, filtered_model_predictions)
print(f"Mean Absolute Error (MAE): {mae:.4f}")

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(filtered_numpy_solutions, filtered_model_predictions)
print(f"Mean Squared Error (MSE): {mse:.4f}")

# Calculate accuracy based on a tolerance threshold
tolerance = 0.1  # Define a tolerance level for accuracy
correct_predictions = np.abs(filtered_numpy_solutions - filtered_model_predictions) <= tolerance
accuracy = np.mean(correct_predictions) * 100
print(f"Accuracy within tolerance of {tolerance}: {accuracy:.2f}%")


Mean Absolute Error (MAE): 1.3789
Mean Squared Error (MSE): 93.1237
Accuracy within tolerance of 0.1: 54.55%


### 1. Mean Absolute Error (MAE): 1.3789
The MAE indicates that, on average, the model’s predictions are about 1.38 units away from the actual solutions (as given by NumPy).  
In the context of root finding, this means the predicted roots tend to have an average error of around 1.38. While this might be acceptable in some cases, it may be considered high depending on the scale and precision required.

### 2. Mean Squared Error (MSE): 93.1237
The MSE shows that the average of the squared errors is approximately 93.12.  
This higher value, compared to MAE, suggests that there are some instances where the model's predictions deviate significantly from the true roots. MSE is particularly sensitive to larger errors, as it squares the differences, highlighting that some predictions are notably inaccurate.

### 3. Accuracy within Tolerance of 0.1: 54.55%
This metric indicates that 54.55% of the model's predictions fall within 0.1 units of the true root values.  
An accuracy of about 54.55% within a very small tolerance suggests that slightly over half of the model's predictions are relatively close to the true values. However, it also indicates that nearly half of the predictions are more than 0.1 units away, which may be significant depending on the application.

# Final thoughts 

The model is reasonably accurate, but there is room for improvement:
- The MAE and MSE values imply that while the model is generally predicting in the correct range, it often makes errors that are non-negligible, as reflected in the relatively high MSE.
- The accuracy within the tolerance suggests that the model achieves close predictions about half of the time, but might need more tuning, additional training data, or a more complex model architecture to improve its precision.

If more accuracy is required, consider further training with additional data or tuning the network parameters to improve performance.
